<a href="https://colab.research.google.com/github/saramurielf/trabajo_final/blob/main/authors/fdabrandao/quick-start/pandasdiet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [201]:
# Install dependencies
%pip install -q amplpy pandas numpy

In [202]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [203]:
%%writefile modelo.mod
# Modelo de transporte de gasolina y gas

set P;  # Plantas de producción
set D;  # Centros de distribución
set R;  # Regiones de demanda
set K;  # Tipos de productos (g: gasolina, s: gas)

param tarifa {K};  # Tarifa por producto en USD/km·hl
param dist_PD {P, D};  # Distancia entre planta y centro (km)
param dist_DR {D, R};  # Distancia entre centro y región (km)

param demanda {R, K};  # Demanda por región y producto (hl)
param capacidad_planta {P, K};  # Capacidad de producción por planta y producto
param capacidad_centro {D};  # Capacidad total de cada centro (hl)

# Variables de decisión
var X {P, D, K} >= 0;  # Volumen enviado de planta a centro
var Y {D, R, K} >= 0;  # Volumen enviado de centro a región

# Función objetivo: minimizar el costo total de transporte
minimize CostoTotal:
    sum {p in P, d in D, k in K} tarifa[k] * dist_PD[p,d] * X[p,d,k]
  + sum {d in D, r in R, k in K} tarifa[k] * dist_DR[d,r] * Y[d,r,k];

# Restricción 1: Satisfacer demanda en cada región por producto
subject to SatisfacerDemanda {r in R, k in K}:
    sum {d in D} Y[d,r,k] = demanda[r,k];

# Restricción 2: No exceder capacidad de planta
subject to CapacidadPlanta {p in P, k in K}:
    sum {d in D} X[p,d,k] <= capacidad_planta[p,k];

# Restricción 3: No exceder capacidad total del centro
subject to CapacidadCentro {d in D}:
    sum {r in R, k in K} Y[d,r,k] <= capacidad_centro[d];

# Restricción 4: Balance de flujo por producto en cada centro
subject to BalanceCentro {d in D, k in K}:
    sum {p in P} X[p,d,k] = sum {r in R} Y[d,r,k];

Overwriting modelo.mod


In [204]:
%%writefile datos.dat
data;

# Conjuntos
set P := P1 P2;
set D := D1 D2 D3;
set R := R1 R2 R3;
set K := g s;

# Tarifas (USD/km·hl)
param tarifa :=
    g 0.67
    s 0.82;

# Distancia planta–centro (km)
param dist_PD : D1 D2 D3 :=
    P1   120  200  150
    P2   100  180  170;

# Distancia centro–región (km)
param dist_DR : R1 R2 R3 :=
    D1   80   90  100
    D2   70   60   85
    D3   60   75   95;

# Demanda por región y producto (hl)
param demanda :=
    [R1, *]  g 500   s 300
    [R2, *]  g 400   s 200
    [R3, *]  g 600   s 250;

# Capacidad de planta por producto (hl)
param capacidad_planta :=
    [P1, *]  g 1000  s 500
    [P2, *]  g 800   s 600;

# Capacidad de operación por centro (hl)
param capacidad_centro :=
    D1 1500
    D2 1200
    D3 1000;

Overwriting datos.dat


In [205]:
ampl.read('modelo.mod');
ampl.read_data('datos.dat');
ampl.option['solver'] = 'highs';
ampl.solve();

HiGHS 1.10.0: HiGHS 1.10.0: optimal solution; objective 325485
20 simplex iterations
0 barrier iterations


In [206]:
# Mostrar el costo total óptimo
print(f"Costo Total Óptimo: {ampl.get_objective('CostoTotal').value()}")

# Mostrar los volúmenes enviados desde las plantas a los centros de distribución
print("\nVolúmenes enviados desde Plantas a Centros de Distribución (X[p, d, k]):")
for p in ampl.get_set('P').members():
    for d in ampl.get_set('D').members():
        for k in ampl.get_set('K').members():
            # Mostrar solo valores significativos, no valores cercanos a cero
            if ampl.get_variable('X')[p, d, k].value() > 1e-6:
                print(f"X[{p},{d},{k}] = {ampl.get_variable('X')[p, d, k].value()}")

# Mostrar los volúmenes enviados desde los centros de distribución a las regiones
print("\nVolúmenes enviados desde Centros de Distribución a Regiones (Y[d, r, k]):")
for d in ampl.get_set('D').members():
    for r in ampl.get_set('R').members():
        for k in ampl.get_set('K').members():
            # Mostrar solo valores significativos, no valores cercanos a cero
            if ampl.get_variable('Y')[d, r, k].value() > 1e-6:
                print(f"Y[{d},{r},{k}] = {ampl.get_variable('Y')[d, r, k].value()}")

Costo Total Óptimo: 325485.0

Volúmenes enviados desde Plantas a Centros de Distribución (X[p, d, k]):
X[P1,D1,g] = 100.0
X[P1,D3,g] = 600.0
X[P1,D3,s] = 150.0
X[P2,D1,g] = 800.0
X[P2,D1,s] = 600.0

Volúmenes enviados desde Centros de Distribución a Regiones (Y[d, r, k]):
Y[D1,R1,s] = 150.0
Y[D1,R2,g] = 300.0
Y[D1,R2,s] = 200.0
Y[D1,R3,g] = 600.0
Y[D1,R3,s] = 250.0
Y[D3,R1,g] = 500.0
Y[D3,R1,s] = 150.0
Y[D3,R2,g] = 100.0
